## **1. Setup**

The project requires several packages that need to be installed into Workspace:

- Langchain: is a framework for developing generative Al applications.
- yt_dip: lets you download YouTube vide
- tiktoken: converts text into tokens.
- docarray: makes it easier to work with multi-model data (in this case mixing audio and text).

In [1]:
!pip install -r requirements.txt

In [2]:
!pip install openai

In [3]:
!pip install python-dotenv


In [4]:
!pip install -U langchain-community

In [6]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu
     ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
     ---------------------------------------- 0.0/536.2 kB ? eta -:--:--
     ------------------- -------------------- 262.1/536.2 kB ? eta -:--:--
     -------------------------------------- 536.2/536.2 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/215.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/215.1 MB ? eta -:--:--
   ---------------------------------------- 0.3/215.1 MB ? eta -:--:--
   ---------------------------------------- 0.5/215.1 MB 1.2 MB/s eta 0:02:59
   ---------------------------------------- 0.8/215.1 MB 1.4 MB/s eta 0:02:34
   ---------------------------------------- 1.0/215.1 MB 1.4 MB/s eta 0:02:33
   ---------------------------------------- 1.6/215.1 MB 1.5 MB/s eta 0:02:20
   ---------------------------------------- 

In [7]:
!pip install transformers soundfile langchain docarray


  Using cached transformers-4.51.3-py3-none-any.whl.metadata (38 kB)
  Using cached huggingface_hub-0.30.2-py3-none-any.whl.metadata (13 kB)
  Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
  Using cached safetensors-0.5.3-cp38-abi3-win_amd64.whl.metadata (3.9 kB)
  Using cached cffi-1.17.1-cp312-cp312-win_amd64.whl.metadata (1.6 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
Using cached transformers-4.51.3-py3-none-any.whl (10.4 MB)
Using cached huggingface_hub-0.30.2-py3-none-any.whl (481 kB)
Using cached tokenizers-0.21.1-cp39-abi3-win_amd64.whl (2.4 MB)
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   -------------------- ------------------- 0.5/1.0 MB 1.9 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 2.5 MB/s eta 0:00:00
Using cached cffi-1.17.1-cp312-cp312-win_amd64.whl (181 kB)
Using cached safetensors-0.5.3-cp3

In [ ]:
import yt_dlp as youtube_dl
from yt_dlp import DownloadError
import os
import glob
import torch
import soundfile as sf
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.agents import Tool, initialize_agent
from langchain.memory import ConversationBufferMemory
from langchain.memory import InMemoryChatMessageHistory

c:\Users\007T\OneDrive\Desktop\ironhack2025\ai-bootcamp-final-project\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### **YouTube Audio Extraction & MP3 Conversion**
This snippet specifies an output folder and a target YouTube URL, then configures yt_dlp to:

1. Download the highest-quality audio stream available.

2. Use FFmpeg to extract and convert that stream into a 192 kbps MP3.

3. Name the resulting file after the video’s title.

4. Enable verbose logging so you can see detailed progress.

It wraps the download call in a try/except block to catch and report any DownloadError that might occur.

In [ ]:
output_dir = "output/"
youtube_url ="https://youtu.be/4h9lQfYLOZU?si=4Z4RCfJjaAdAp2e1"

In [ ]:
# Configuration for yt_dlp
def download_audio(url: str) -> str:
    
    ydl_config = {
        "format": "bestaudio/best",
        # List of post-processing steps; each dict represents one processor
        "postprocessors": [
            {
                "key": "FFmpegExtractAudio",   # Extract audio from the downloaded file
                "preferredcodec": "mp3",       # Convert audio to MP3 format
                "preferredquality": "192"      # Set audio quality to 192 kbps
            }
        ],
        # Template for naming the output file: "<video title>.<extension>"
        "outtmpl": "output/%(title)s.%(ext)s",
        "verbose": True  # Enable detailed logging
    }

    try:
        # Initialize the downloader with the specified config and start download
        with youtube_dl.YoutubeDL(ydl_config) as ydl:
            info = ydl.extract_info(url, download=True)
    except DownloadError as e:
        # Print any download errors that occur
        print("DownloadError:", e)
    
    return f"Downloaded: output_audio/{info['id']}.mp3"

[debug] Encodings: locale cp1252, fs utf-8, pref cp1252, out UTF-8 (No VT), error UTF-8 (No VT), screen UTF-8 (No VT)
[debug] yt-dlp version stable@2025.03.31 from yt-dlp/yt-dlp [5e457af57] (pip) API
[debug] params: {'format': 'bestaudio/best', 'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'mp3', 'preferredquality': '192'}], 'outtmpl': 'output/%(title)s.%(ext)s', 'verbose': True, 'compat_opts': set(), 'http_headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.54 Safari/537.36', 'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8', 'Accept-Language': 'en-us,en;q=0.5', 'Sec-Fetch-Mode': 'navigate'}}
[debug] Python 3.12.0 (CPython AMD64 64bit) - Windows-10-10.0.19045-SP0 (OpenSSL 3.0.11 19 Sep 2023)
[debug] exe versions: ffmpeg 2024-02-01-git-94422871fc-full_build-www.gyan.dev (setts), ffprobe 2024-02-01-git-94422871fc-full_build-www.gyan.dev
[debug] Optional libraries: certi

[debug] Plugin directories: none
[debug] Loaded 1850 extractors


[youtube] Extracting URL: https://youtu.be/4h9lQfYLOZU?si=4Z4RCfJjaAdAp2e1
[youtube] 4h9lQfYLOZU: Downloading webpage
[youtube] 4h9lQfYLOZU: Downloading tv client config
[youtube] 4h9lQfYLOZU: Downloading player fded239a-main
[youtube] 4h9lQfYLOZU: Downloading tv player API JSON
[youtube] 4h9lQfYLOZU: Downloading ios player API JSON


[debug] [youtube] Decrypted nsig C-tYsz24fYW1sp => 0zf-okR24S1XQA
[debug] Saving youtube-nsig.fded239a-main to cache
[debug] [youtube] Decrypted nsig 0_A-ueP0p0-8T8 => u-QLso9KSKTzbA
[debug] [youtube] 4h9lQfYLOZU: ios client https formats require a GVS PO Token which was not provided. They will be skipped as they may yield HTTP Error 403. You can manually pass a GVS PO Token for this client with --extractor-args "youtube:po_token=ios.gvs+XXX". For more information, refer to  https://github.com/yt-dlp/yt-dlp/wiki/PO-Token-Guide . To enable these broken formats anyway, pass --extractor-args "youtube:formats=missing_pot"


[youtube] 4h9lQfYLOZU: Downloading m3u8 information


...

[download] Destination: output\IELTS Vocabulary Synonyms for the 40 Most Commonly Used Words.webm
[download] 100% of   20.65MiB in 00:02:14 at 157.62KiB/s 


[debug] ffmpeg command line: ffprobe -show_streams "file:output\IELTS Vocabulary Synonyms for the 40 Most Commonly Used Words.webm"


[ExtractAudio] Destination: output\IELTS Vocabulary Synonyms for the 40 Most Commonly Used Words.mp3


[debug] ffmpeg command line: ffmpeg -y -loglevel repeat+info -i "file:output\IELTS Vocabulary Synonyms for the 40 Most Commonly Used Words.webm" -vn -acodec libmp3lame -b:a 192.0k -movflags +faststart "file:output\IELTS Vocabulary Synonyms for the 40 Most Commonly Used Words.mp3"


Deleting original file output\IELTS Vocabulary Synonyms for the 40 Most Commonly Used Words.webm (pass -k to keep)


### **Batch Transcription of MP3 Files with Whisper**

1. Detects whether to run on GPU (CUDA) or CPU and sets the optimal PyTorch data type for memory efficiency.

2. Loads the openai/whisper-large-v3 model (using safetensors for faster, lower-memory loading) and moves it to the chosen device.

3. Loads the accompanying processor (tokenizer + feature extractor).

4. Builds an ASR pipeline around that model + processor.

5. Uses glob to collect all .mp3 files in the specified output_dir and validates that at least one file exists.

6. Defines a single transcript output path (files/transcripts/transcript.txt) and ensures its folder is created.

7. Iterates over each MP3, reads the audio into an array, runs the ASR pipeline to generate text, and writes the resulting transcript (overwriting on each loop) to the designated text file.

In [ ]:
# 1. Figure out if I can use GPU
device = "cuda:0" if torch.cuda.is_available() else "cpu"
#    and choose torch dtype to save memory on GPU
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

# 2. Pick the Whisper model I want
model_id = "openai/whisper-large-v3"

# 3. Load the model weights (using safetensors to speed things up)
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True
)
# move the model to the right device
model.to(device)


In [ ]:
# 4. Load the processor that has both tokenizer & feature extractor
processor = AutoProcessor.from_pretrained(model_id)


In [ ]:
# 5. Build the ASR pipeline with my model and processor
pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
)

In [ ]:
# 6. Find all the MP3 files in my output directory
audio_files = glob.glob(os.path.join(output_dir, "*.mp3"))

# 7. Make sure I actually have files to process
if not audio_files:
    raise ValueError(f"No .mp3 files found in {output_dir}")

# 8. Loop through each file, transcribe, and save
# Where to save the text transcript
output_file  = "files/transcripts/transcript.txt"

for audio_path in audio_files:
    print(f"Processing {audio_path}...")
    #    read the audio array and sampling rate
    audio_array, sr = sf.read(audio_path)
    sample = {"array": audio_array, "sampling_rate": sr}

    #    run the pipeline to get the transcript
    result = pipe(sample)
    text = result["text"]

    #    prepare output path & ensure folder exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    #    write the transcript to disk
    with open(output_file, "w", encoding="utf-8") as f:
        f.write(text)

    print(f"Saved transcript to {output_file}\n")

In [ ]:
"""from dotenv import load_dotenv
# Load .env into environment
_ = load_dotenv()

# Initialize the new OpenAI client
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# —————— Transcription ——————
print("Converting audio to text…")


with open(audio_filename, "rb") as f:
    # Call the new transcription endpoint
    transcription = client.audio.transcriptions.create(
        file=f,            # the binary audio file
        model="whisper-1"  # the current Whisper model name
    )

# Extract the plain text
text = transcription.text

# —————— Save to disk ——————
# Ensure the output folder exists
os.makedirs(os.path.dirname(output_file), exist_ok=True)

with open(output_file, "w", encoding="utf-8") as out:
    out.write(text)

print(f"Transcript saved to {output_file}")"""

### **Load Transcript File into LangChain Documents** 

Uses LangChain’s TextLoader to read transcript.txt and convert it into a list of Document objects for downstream processing.

In [ ]:
# Import the TextLoader class from the Langchain.docunent_loaders nodule
from langchain.document_loaders import TextLoader

# Create a new instance of the TextLoader class, specifying the directory containing the text files
loader = TextLoader("./files/transcripts/transcript.txt")

# Load the documents from the specified directory using the TextLoader instance
docs = loader.load()

In [ ]:
docs[0]

Document(metadata={'source': './files/transcripts/transcript.txt'}, page_content="Hi everyone, Chris here from IELTSadvantage.com with another lesson and today what we're gonna focus on is how to practice IELTS listening. So what we're gonna do is look at why doing lots of practice tests is a terrible idea. In fact, this is the worst thing you could do. If you think that just doing lots and lots and lots of practice tests is gonna help you get a higher score, you are wrong. But what then I'm gonna show you is three better ways to practice that will actually improve your scores because at the end of the day, what are we doing? We're helping you improve your scores. So we're only gonna teach you the things that work and make you aware of the things that don't work. And these three ways are totally free and you can do them at home by yourself without a teacher. So they're absolutely brilliant. So number one, why doing lots of practice tests is a terrible idea. Well, the first thing is the

### **Build & Configure the RetrievalQA Pipeline**

1. Imports the core RAG components—RetrievalQA (the QA chain), ChatOpenAI (the LLM wrapper), DocArrayInMemorySearch (an in-memory vector store), and OpenAIEmbeddings (the embeddings generator).



In [ ]:
#Import the RetrievalQA class from the langchain.chai module
from langchain.chains import RetrievalQA

#Import the ChatOpenAI class fron the Langchain.chat_models module
from langchain.chat_models import ChatOpenAI

# Import the DocArrayinMenorySearch class from the Langchain.vectorstores module
from langchain.vectorstores import DocArrayInMemorySearch

# Import the OpenAIEnbeddings class from the Langchain.embeddings module
from langchain.embeddings import OpenAIEmbeddings

2. Creates a DocArrayInMemorySearch index from your docs, embedding each with OpenAI’s embeddings API.



In [ ]:
#Create a new DoCArrayInMenorySearch Instance from the Specified documents and embeddings
db = DocArrayInMemorySearch.from_documents(
docs,
OpenAIEmbeddings())

C:\Users\007T\AppData\Local\Temp\ipykernel_25060\3091044111.py:4: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  OpenAIEmbeddings())
c:\Users\007T\OneDrive\Desktop\ironhack2025\ai-bootcamp-final-project\.venv\Lib\site-packages\pydantic\_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


3. Converts that index into a retriever for semantic search.

4. Instantiates a ChatOpenAI model with zero temperature for deterministic responses.

In [ ]:
#Convert DocArrayInMemorySearch instance to a retriever
retriever = db.as_retriever()

#Create a new chatOpenAi 
llm = ChatOpenAI(temperature = 0.0)

C:\Users\007T\AppData\Local\Temp\ipykernel_25060\2616038350.py:5: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature = 0.0)




5. Builds a RetrievalQA chain (using the “stuff” strategy) that ties together the LLM and retriever, with verbose=True to print intermediate debug info.

In [ ]:
# Create a new RetrievalQA instance with the specified parameters
qa_stuff = RetrievalQA.from_chain_type(
llm=llm,
chain_type="stuff",
retriever= retriever,
verbose=True)

### **Multi-Tool Conversational Agent with Memory**

Sets up three custom tools (transcript Q&A, summarization, and YouTube audio download), configures a conversation buffer to remember past messages, and initializes a LangChain agent that can use these tools interactively while preserving chat history.

In [ ]:
#Create a tool to answer questions about my audio transcripts
qa_tool = Tool(
    name="TranscriptQA",
    func=qa_stuff.run,
    description="Answer questions based on the audio transcripts."
)

In [ ]:
#define a summarization function and wrap it as a tool
def summarize_transcript(text: str) -> str:
    return qa_stuff.run(f"Summarize this:\n\n{text}")

summarizer_tool = Tool(
    name="TranscriptSummarizer",
    func=summarize_transcript,
    description="Generate a concise summary of a given transcript text."
)

summarizer_tool = Tool(
    name="TranscriptSummarizer",
    func=summarize_transcript,
    description="Generate a concise summary of a given transcript text."
)

In [ ]:
#et up a downloader tool to grab YouTube audio and convert it to MP3
downloader_tool = Tool(
    name="YouTubeDownloader",
    func=download_audio,
    description="Download and convert a YouTube URL to an MP3 file."
)


In [ ]:
session_store = {}

def get_session_history(session_id: str) -> InMemoryChatMessageHistory:
    """Ensures each session has its own message history"""
    if session_id not in session_store:
        session_store[session_id] = InMemoryChatMessageHistory()
    return session_store[session_id]

In [ ]:
import uuid
session_id = str(uuid.uuid4())  # e.g. "4f9b8a2e-1c3d-4f5a-9e6b-7d8f0a1b2c3d"

#configure a memory buffer to keep the full conversation history
memory = get_session_history(session_id)

In [ ]:
# I initialize the agent with my tools, the LLM, and the memory buffer
agent = initialize_agent(
    tools=[qa_tool, summarizer_tool, downloader_tool],
    llm=llm,
    agent="ConversationBufferMemory",
    memory=memory,
    verbose=True
)

### **Execute RetrievalQA Query & Display Answer**
1. Defines the user’s question as the query string.

2. Runs that query through the previously configured RetrievalQA chain (qa_stuff), which retrieves relevant passages and then generates an answer.

3. Prints out the final response text to the console.

In [ ]:
# set the query
query = "What is this video about?"

#run the query 
response = qa_stuff.run(query)

#print response
print(response)


C:\Users\007T\AppData\Local\Temp\ipykernel_25060\634586374.py:5: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_stuff.run(query)




> Entering new RetrievalQA chain...

> Finished chain.
This video is about how to practice IELTS listening effectively. It discusses why just doing lots of practice tests is not the best approach and offers three better ways to practice that can actually improve your scores. The video emphasizes the importance of listening for pleasure, listening actively, and listening reflectively to enhance your listening skills and ultimately improve your IELTS test performance.


In [ ]:
query = "what is the three better ways to practice IELTS listening"

#run the query 
response = qa_stuff.run(query)

#print response
print(response)



> Entering new RetrievalQA chain...

> Finished chain.
The three better ways to practice IELTS listening are:

1. Listen for pleasure: Listen to things you enjoy consistently, such as podcasts, TV shows, movies, radio, TED Talks, YouTube, and real people.
2. Listen actively: Focus on one or two things while listening, such as vocabulary, accent, intonation, or multiple speakers. Actively engage with the material to improve your skills.
3. Listen reflectively: Take genuine practice tests under exam conditions, evaluate your mistakes honestly, identify patterns in your errors, and take action to improve on specific weaknesses.
